In [1]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

___

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader # load batches to the network

from feito.basecaller_trainer import BasecallerTrainer as Trainer
from feito.models import SimpleNet, Rodan
from feito.loss_functions import ctc_label_smoothing_loss
from feito.dataloader import DatasetONT
from feito.callbacks import CSVLogger

/home/javila/micromamba/envs/basecalling-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


___
PARAMS

In [3]:
EPOCHS=5
BATCH_SIZE=16

### Training network
Model, loss function and optimizer

In [4]:
# network to use
model=SimpleNet()
# model=Rodan()
model_output_len = model.output_len # another way to obtain the output of the model https://github.com/biodlab/RODAN/blob/029f7d5eb31b11b53537f13164bfedee0c0786e4/model.py#L317
loss_fn = nn.CTCLoss() #ctc_label_smoothing 
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)


Load dataset

In [5]:
# dataset
dataset_train = DatasetONT(recfile="../data/subsample_train.hdf5", output_network_len=model_output_len) #simplenet_output_len)
dataset_val   = DatasetONT(recfile="../data/subsample_val.hdf5", output_network_len=model_output_len) #simplenet_output_len)

dataloader_train = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

Callbacks

In [6]:
csv_logger=CSVLogger(list_vars=["epoch","train_loss","val_loss"], out_file="output/training/metrics.csv", overwrite=True)

Train

In [7]:
trainer=Trainer(
    model=SimpleNet(),
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    train_loader=dataloader_train,
    validation_loader=dataloader_val,
    criterion=loss_fn,
    optimizer=optimizer,
    callbacks=[csv_logger]
)

In [8]:
trainer.fit(epochs=EPOCHS)

Training Epoch: 1 | Batch: 0/13:   0%|                                                     | [00:00][W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Training Epoch: 1 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.2345]
Validate Epoch: 1 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.2150]


calling callback


Training Epoch: 2 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.1579]
Validate Epoch: 2 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.2150]


calling callback


Training Epoch: 3 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.1487]
Validate Epoch: 3 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.2150]


calling callback


Training Epoch: 4 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.2120]
Validate Epoch: 4 | Batch: 12/13: 100%|███████████████████████████████████████| [00:02, loss=2.2150]


calling callback


Training Epoch: 5 | Batch: 12/13: 100%|██████████████████████████████████████████| [00:04, loss=inf]
Validate Epoch: 5 | Batch: 12/13: 100%|███████████████████████████████████████| [00:02, loss=2.2150]

calling callback
Done!


In [9]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [10]:
torch.cuda.is_available()

False